#### Análise Exploratória da Base de Dados

In [24]:
import pandas as pd
from scipy.stats import shapiro

In [6]:
df = pd.read_csv("../Data/casas_londres_traduzido.csv")

In [7]:
df.head()

,Endereco,Bairro,Quartos,Banheiros,Metros Quadrados,Idade Predio,Jardim,Garagem,Andares,Tipo Imovel,Tipo Aquecimento,Varanda,Estilo Interior,Vista,Materiais,Estado Predio,Preco (£)
0,78 Regent Street,Notting Hill,2,3,179,72,No,No,3,Semi-Detached,Electric Heating,High-level Balcony,Industrial,Garden,Marble,Renovated,2291200
1,198 Oxford Street,Westminster,2,1,123,34,Yes,No,1,Apartment,Central Heating,High-level Balcony,Industrial,City,Laminate Flooring,Old,1476000
2,18 Regent Street,Soho,5,3,168,38,No,Yes,3,Semi-Detached,Central Heating,No Balcony,Industrial,Street,Wood,Renovated,1881600
3,39 Piccadilly Circus,Islington,5,1,237,53,Yes,Yes,1,Apartment,Underfloor Heating,No Balcony,Classic,Park,Granite,Renovated,1896000
4,116 Fleet Street,Marylebone,4,1,127,23,No,Yes,2,Semi-Detached,Central Heating,No Balcony,Modern,Park,Wood,Old,1524000


In [8]:
print(f"Número de linhas: {df.shape[0]}; Número de colunas: {df.shape[1]}")

Número de linhas: 1000; Número de colunas: 17


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Endereco          1000 non-null   object
 1   Bairro            1000 non-null   object
 2   Quartos           1000 non-null   int64 
 3   Banheiros         1000 non-null   int64 
 4   Metros Quadrados  1000 non-null   int64 
 5   Idade Predio      1000 non-null   int64 
 6   Jardim            1000 non-null   object
 7   Garagem           1000 non-null   object
 8   Andares           1000 non-null   int64 
 9   Tipo Imovel       1000 non-null   object
 10  Tipo Aquecimento  1000 non-null   object
 11  Varanda           1000 non-null   object
 12  Estilo Interior   1000 non-null   object
 13  Vista             1000 non-null   object
 14  Materiais         1000 non-null   object
 15  Estado Predio     1000 non-null   object
 16  Preco (£)         1000 non-null   int64 
dtypes: int64(6), ob

In [10]:
df.isnull().sum()

Endereco            0
Bairro              0
Quartos             0
Banheiros           0
Metros Quadrados    0
Idade Predio        0
Jardim              0
Garagem             0
Andares             0
Tipo Imovel         0
Tipo Aquecimento    0
Varanda             0
Estilo Interior     0
Vista               0
Materiais           0
Estado Predio       0
Preco (£)           0
dtype: int64

In [15]:
contagem_nulos = df.isnull().sum()
if contagem_nulos.sum() == 0:
    print("Não há valores nulos no DataFrame.")
else:
    print("Colunas com valores nulos:")
    print(contagem_nulos[contagem_nulos > 0])

Não há valores nulos no DataFrame.


#### Estatística

In [17]:
df_num = df.select_dtypes(include=['int64'])

In [22]:
Q1 = df_num.quantile(0.25)
Q3 = df_num.quantile(0.75)
IQR = Q3 - Q1

descricao_transposta = df_num.describe().T
estatisticas_adicionais = pd.DataFrame({
    'skew': df_num.skew(),
    'kurtosis': df_num.kurtosis(),
    'range': df_num.max() - df_num.min(),
    'valores_unicos': df_num.nunique(),
    'outliers': ((df_num < (Q1 - 1.5 * IQR)) | (df_num > (Q3 + 1.5 * IQR))).sum()
})
descricao_aprimorada = pd.concat([descricao_transposta, estatisticas_adicionais], axis=1)

In [23]:
descricao_aprimorada = descricao_aprimorada.round(2)
descricao_aprimorada

,count,mean,std,min,25%,50%,75%,max,skew,kurtosis,range,valores_unicos,outliers
Quartos,1000.0,3.01,1.39,1.0,2.0,3.0,4.0,5.0,-0.02,-1.26,4,5,0
Banheiros,1000.0,2.00,0.81,1.0,1.0,2.0,3.0,3.0,-0.01,-1.47,2,3,0
Metros Quadrados,1000.0,149.63,58.06,50.0,99.0,148.0,201.0,249.0,0.03,-1.25,199,197,0
Idade Predio,1000.0,49.96,29.07,1.0,24.0,50.0,76.0,99.0,-0.02,-1.26,98,99,0
Andares,1000.0,1.99,0.82,1.0,1.0,2.0,3.0,3.0,0.02,-1.52,2,3,0
Preco (£),1000.0,1840807.28,879348.41,386666.0,1161400.0,1721999.0,2390500.0,4980000.0,0.65,-0.07,4593334,786,10


In [34]:
for coluna in df_num.columns:
    if df_num[coluna].notnull().sum() > 2: 
        stat, p = shapiro(df_num[coluna])  
        print(f"\nColuna: {coluna}")
        print(f"Shapiro-Wilk Teste: Estatística={stat:.3f}, p={p:.3f}")
        if p > 0.05:
            print("Os dados seguem uma distribuição normal.")
        else:
            print("Os dados NÃO seguem uma distribuição normal.")
    else:
        print(f"\nColuna: {coluna}")
        print("A coluna possui poucos valores para o teste de normalidade.")


Coluna: Quartos
Shapiro-Wilk Teste: Estatística=0.892, p=0.000
Os dados NÃO seguem uma distribuição normal.

Coluna: Banheiros
Shapiro-Wilk Teste: Estatística=0.796, p=0.000
Os dados NÃO seguem uma distribuição normal.

Coluna: Metros Quadrados
Shapiro-Wilk Teste: Estatística=0.951, p=0.000
Os dados NÃO seguem uma distribuição normal.

Coluna: Idade Predio
Shapiro-Wilk Teste: Estatística=0.948, p=0.000
Os dados NÃO seguem uma distribuição normal.

Coluna: Andares
Shapiro-Wilk Teste: Estatística=0.791, p=0.000
Os dados NÃO seguem uma distribuição normal.

Coluna: Preco (£)
Shapiro-Wilk Teste: Estatística=0.961, p=0.000
Os dados NÃO seguem uma distribuição normal.


#### Colunas Categóricas

In [35]:
df_cat = df.select_dtypes(include=['object'])

In [40]:
for coluna in df_cat:
    print(f"\nColuna: {df[coluna].value_counts()}")


Coluna: Endereco
58 Fleet Street          4
96 King's Road           4
91 King's Road           4
96 Fleet Street          3
151 Regent Street        3
                        ..
124 King's Road          1
165 Park Lane            1
99 Camden High Street    1
155 Park Lane            1
152 Baker Street         1
Name: count, Length: 770, dtype: int64

Coluna: Bairro
Kensington      114
Marylebone      113
Camden          106
Westminster      98
Greenwich        97
Islington        97
Notting Hill     96
Soho             96
Chelsea          94
Shoreditch       89
Name: count, dtype: int64

Coluna: Jardim
Yes    512
No     488
Name: count, dtype: int64

Coluna: Garagem
Yes    501
No     499
Name: count, dtype: int64

Coluna: Tipo Imovel
Detached House    349
Semi-Detached     345
Apartment         306
Name: count, dtype: int64

Coluna: Tipo Aquecimento
Electric Heating      266
Gas Heating           255
Underfloor Heating    243
Central Heating       236
Name: count, dtype: int64

Colun

In [41]:
for coluna in df_cat:
    freq_cumulativa = df[coluna].value_counts().cumsum()
    print(f"\nColuna: {freq_cumulativa}")


Coluna: Endereco
58 Fleet Street             4
96 King's Road              8
91 King's Road             12
96 Fleet Street            15
151 Regent Street          18
                         ... 
124 King's Road           996
165 Park Lane             997
99 Camden High Street     998
155 Park Lane             999
152 Baker Street         1000
Name: count, Length: 770, dtype: int64

Coluna: Bairro
Kensington       114
Marylebone       227
Camden           333
Westminster      431
Greenwich        528
Islington        625
Notting Hill     721
Soho             817
Chelsea          911
Shoreditch      1000
Name: count, dtype: int64

Coluna: Jardim
Yes     512
No     1000
Name: count, dtype: int64

Coluna: Garagem
Yes     501
No     1000
Name: count, dtype: int64

Coluna: Tipo Imovel
Detached House     349
Semi-Detached      694
Apartment         1000
Name: count, dtype: int64

Coluna: Tipo Aquecimento
Electric Heating       266
Gas Heating            521
Underfloor Heating     764
Centr

In [46]:
for coluna in df_cat:
    proporcao = df[coluna].value_counts(normalize=True) * 100
    print(f"\nColuna: {proporcao}")  


Coluna: Endereco
58 Fleet Street          0.4
96 King's Road           0.4
91 King's Road           0.4
96 Fleet Street          0.3
151 Regent Street        0.3
                        ... 
124 King's Road          0.1
165 Park Lane            0.1
99 Camden High Street    0.1
155 Park Lane            0.1
152 Baker Street         0.1
Name: proportion, Length: 770, dtype: float64

Coluna: Bairro
Kensington      11.4
Marylebone      11.3
Camden          10.6
Westminster      9.8
Greenwich        9.7
Islington        9.7
Notting Hill     9.6
Soho             9.6
Chelsea          9.4
Shoreditch       8.9
Name: proportion, dtype: float64

Coluna: Jardim
Yes    51.2
No     48.8
Name: proportion, dtype: float64

Coluna: Garagem
Yes    50.1
No     49.9
Name: proportion, dtype: float64

Coluna: Tipo Imovel
Detached House    34.9
Semi-Detached     34.5
Apartment         30.6
Name: proportion, dtype: float64

Coluna: Tipo Aquecimento
Electric Heating      26.6
Gas Heating           25.5
Underflo